<a href="https://colab.research.google.com/github/1eetnals/Code-similarity/blob/main/(huggingface)Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip data/open.zip -d data/
!pip install transformers datasets

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 325 kB 27.2 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 84.9 MB/s 
     |████████████████████████████████| 6.6 MB 83.8 MB/s 
     |████████████████████████████████| 880 kB 64.4 MB/s 
     |████████████████████████████████| 1.1 MB 64.6 MB/s 
     |████████████████████████████████| 212 kB 74.9 MB/s 
     |████████████████████████████████| 136 kB 78.5 MB/s 
     |████████████████████████████████| 127 kB 93.1 MB/s 
     |████████████████████████████████| 271 kB 95.6 MB/s 
     |████████████████████████████████| 144 kB 100.6 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=09a3dc7fdb271dc8e07232402073a9b30475e3a67eb0a17dfb1b89315118493a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d6602

In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric
import torch


MODEL = "klue/roberta-base"
INPUT = "/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/sample_train.csv"
MAX_LEN = 512  #원래 : 256
dataset = load_dataset("csv", data_files=INPUT)['train']

Using custom data configuration default-ef99256902558b87


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ef99256902558b87/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#csv 형식의 training 데이터를 로드합니다.
import pandas as pd 

train = pd.read_csv("/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/sample_train.csv")
c1 = train.code1 #training 데이터에서 문서 추출
c2 = train.code2 #training 데이터에서 라벨 추출
s = train.similar #training 데이터에서 라벨 추출

In [ ]:
import re 

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 

        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
        review = re.sub(r'\d+','', review)#숫자 제거
        review = review.lower() #소문자 변환
        #for i in ['\n'] :
          #new_id = new_id.replace(i, "")
        #review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        #review = re.sub(r'\s+', ' ', review) #spaces 제거
        #review = re.sub(r"^\s+", '', review) #space from start 제거
        #review = re.sub(r'\s+$', '', review) #space from the end 제거
        #review = re.sub(r'_', ' ', review) #space from the end 제거
        corpus.append(review) 
        
    return corpus

In [ ]:
temp = clean_text(train['code1']) #메소드 적용
train['code1'] = temp
print(train)

temp = clean_text(train['code2']) #메소드 적용
train['code2'] = temp
print(train)

                                                   code1  \
0      flag gocnt while flag go cnt x intinput if x f...   
1                          b c mapint inputsplitprintb c   
2      import numpy as npimport sysread sysstdinbuffe...   
3                          b c mapint inputsplitprintb c   
4      sinputtinputansfor i in rangelens if siti ansp...   
...                                                  ...   
17965  import matha b h m mapint inputsplit deg abs h...   
17966  n intinputs t inputsplitans for x in rangen an...   
17967  hwk mapint inputsplit アイテムが何処に何個あるかfs w for i ...   
17968  eimport mathn intinputans i if n tmp i while t...   
17969  for i in range list mapint rawinputsplit x lis...   

                                                   code2  similar  
0      python import sysff opentesttxt rff sysstdincn...        1  
1      import numpy as npn intinputa nparraylistmapin...        0  
2      n m mapint inputsplitif m center m else center...        0  
3      

In [ ]:
train
train.to_csv("/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/preprocess.csv", index=False)


In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric
import torch


MODEL = "klue/roberta-base"
INPUT = "/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/preprocess.csv"
MAX_LEN = 512  #원래 : 256
dataset = load_dataset("csv", data_files=INPUT)['train']
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def example_fn(examples):
    outputs = tokenizer(examples['code1'], examples['code2'], padding=True, max_length=MAX_LEN,truncation=True)
    if 'similar' in examples:
        outputs["labels"] = examples["similar"]
    return outputs

dataset = dataset.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
dataset = dataset.train_test_split(0.1)

_collator = DataCollatorWithPadding(tokenizer=tokenizer)
_metric = load_metric("glue", "sst2")

def metric_fn(p):   #accuracy metrics
    preds, labels = p
    output =  _metric.compute(references=labels, predictions=np.argmax(preds, axis=-1))
    return output
    
model = RobertaForSequenceClassification.from_pretrained(MODEL) # RobertaForSequenceClassification 는 BertForSequenceClassification 와 달리 pooler가 없는게 기본이기 때문에 문장 유사도에 사용 가능.

args = TrainingArguments(
    'runs/',
    per_device_train_batch_size=4,   #원래32
    num_train_epochs=3,  #원래:10
    gradient_accumulation_steps = 16, #    
    per_device_eval_batch_size= 8,  #
    do_train=True,
    do_eval=True,
    save_strategy="epoch",
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    #####
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps = 8,
    fp16 = True,
    dataloader_num_workers = 8,
)

trainer = Trainer(
        model=model,
        args=args,
        data_collator=_collator,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=metric_fn)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


Using custom data configuration default-5ac77f32d661a9c9


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5ac77f32d661a9c9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

...

  0%|          | 0/17970 [00:00<?, ?ex/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier

'cuda'

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16173
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimizati

Epoch,Training Loss,Validation Loss,Accuracy
0,0.541000,0.332651,0.863105
1,0.310300,0.206827,0.906511
2,0.136000,0.128742,0.961603


***** Running Evaluation *****
  Num examples = 1797
  Batch size = 8
Saving model checkpoint to runs/checkpoint-252
Configuration saved in runs/checkpoint-252/config.json
Model weights saved in runs/checkpoint-252/pytorch_model.bin
tokenizer config file saved in runs/checkpoint-252/tokenizer_config.json
Special tokens file saved in runs/checkpoint-252/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
***** Running Evaluation *****
  Num examples = 1797
  Batch size = 8
Saving model checkpoint to runs/checkpoint-504
Configuration saved in runs/checkpoint-504/confi

TrainOutput(global_step=756, training_loss=0.3291123153040649, metrics={'train_runtime': 1491.4512, 'train_samples_per_second': 32.531, 'train_steps_per_second': 0.507, 'total_flos': 8307120045343800.0, 'train_loss': 0.3291123153040649, 'epoch': 3.0})

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/test.csv")
temp = clean_text(test['code1']) #메소드 적용
test['code1'] = temp

temp = clean_text(test['code2']) #메소드 적용
test['code2'] = temp


In [ ]:
test
test.to_csv("/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/test_preprocess.csv", index=False)

In [ ]:
import pandas as pd

TEST = "/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/test_preprocess.csv"
SUB = "/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/sample_submission.csv"

test_dataset = load_dataset("csv", data_files=TEST)['train']
test_dataset = test_dataset.map(example_fn, remove_columns=['code1', 'code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SUB)
df['similar'] = np.argmax(predictions.predictions, axis=-1)
df.to_csv('/content/drive/MyDrive/dacon/코드유사성판단AI경진대회/data/submission1_preprocess.csv', index=False)

Using custom data configuration default-672b44e210dd8b28


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-672b44e210dd8b28/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/179700 [00:00<?, ?ex/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: pair_id. If pair_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 179700
  Batch size = 8
